In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time 

from code.experiment_functions import * 

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import warnings
import numpy as np

# Create Dataset
np.random.seed(0)
X_xor =  np.random.uniform(-1, 1, (40, 4)) #np.random.rand(200, 4)

y_xor = np.logical_xor(X_xor[:, 0] > 0,
                       X_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)

fig = plt.figure(figsize=(10,10))
plt.scatter(X_xor[y_xor == 1, 0],
            X_xor[y_xor == 1, 1],
            c='b', marker='x',
            label='1')
plt.scatter(X_xor[y_xor == -1, 0],
            X_xor[y_xor == -1, 1],
            c='r',
            marker='s',
            label='-1')

plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_xor = np.round(X_xor, 2)
X_xor[X_xor == 0] = 0.01
X_train, X_test, y_train, y_test = train_test_split(X_xor, y_xor, test_size=0.33, random_state=42)

In [ ]:
X_train = [list(x) for x in X_train]
X_test = [list(x) for x in X_test]
y_train = [int(1) if x == 1 else int(0) for x in y_train]
y_test = [int(1) if x == 1 else int(0) for x in y_test]

In [ ]:
"""
with open('results/version6/train_xor.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version6/test_xor.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)
"""

#### load saved data

In [2]:
with open('results/version6/train_xor.json') as json_file:
    X_train = json.load(json_file)[0]

with open('results/version6/train_xor.json') as json_file:
    y_train = json.load(json_file)[1]

In [3]:
with open('results/version6/test_xor.json') as json_file:
    X_test = json.load(json_file)[0]

with open('results/version6/test_xor.json') as json_file:
    y_test = json.load(json_file)[1]

In [ ]:
X_train[0]

## Test

In [ ]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = [0.5, 0.5, 0.5, 0.5],
                           threshold=0.3,
                           lrParameter=0.1, 
                           n_epochs=20, 
                           trainingBias=True,
                           epoch_results=True,
                           phaseEstrategyOperator='angle',
                           trainingApproaches={'hsgs','phase-encoding'})

In [ ]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.3,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=10,
                                   bias=True, 
                                   phaseEstrategyOperator='angle',
                                   testingApproaches={'hsgs', 'phase-encoding'})

In [ ]:
print('phase avg_error', results[0])
print('phase weights', results[2])

print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])

print('phase neuron_outputs: ', results[1])
print('HSGS neuron_outputs: ', results[4])
print('targets: ', results[6])

## Execute Experiment

In [4]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [5]:
# search space

initial_weights = [
 [0.5, 0.5, 0.5, 0.5],
 [0.6778165367962301, 0.27000797319216485, 0.7351940221225949, 0.9621885451174382],
 [0.24875314351995803, 0.5761573344178369, 0.592041931271839, 0.5722519057908734]]


threshold_space = [0.1, 0.3, 0.5, 0.7, 0.8]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original', 'angle', 'radius', 'angleradius']

In [6]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=w,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=50, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs','phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=10,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'hsgs', 'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)

                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(w)
                    print('\n')

lrParameter:  0.02
threshold:  0.1
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  5
best error HSGS training:  7
AVG TEST ERROR HSGS    0.5713999999999999
AVG TEST ERROR PHASE   0.28569999999999995


lrParameter:  0.02
threshold:  0.1
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  4
best error HSGS training:  7
AVG TEST ERROR HSGS    0.5713999999999999
AVG TEST ERROR PHASE   0.48572


lrParameter:  0.1
threshold:  0.1
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  6
best error HSGS training:  7
AVG TEST ERROR HSGS    0.5713999999999999
AVG TEST ERROR PHASE   0.54284


lrParameter:  0.1
threshold:  0.1
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  5
best error HSGS training:  7
AVG TEST ERROR HSGS    0.5713999999999999
AVG TEST ERROR PHASE   0.4286


lrParameter:  0.02
threshold:  0.3
trainingBias:  False
p

In [7]:
experiment_data = pd.DataFrame(output)
experiment_data

,model,phase_strategy,bias,threshold,lr,avg_error,trained_weights,initial_weights,neuron_outputs
0,phase-encoding,original,False,0.1,0.02,0.28570,"[-0.5059999999999999, 0.8688000000000041, 1.72...","[0.5, 0.5, 0.5, 0.5]","[0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, ..."
1,HSGS,original,False,0.1,0.02,0.57140,"[0.120000000000002, 0.3700000000000009, 1.7799...","[0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,phase-encoding,original,True,0.1,0.02,0.48572,"[0.03700000000000137, 0.7018000000000032, 1.74...","[0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
3,HSGS,original,True,0.1,0.02,0.57140,"[0.120000000000002, 0.3700000000000009, 1.7799...","[0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,phase-encoding,original,False,0.1,0.10,0.54284,"[-2.054999999999998, 2.0169999999999937, 7.264...","[0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, ..."
...,...,...,...,...,...,...,...,...,...
475,HSGS,angleradius,True,0.8,0.02,0.57140,"[-0.13124685648004175, 0.44615733441783945, 1....","[0.24875314351995803, 0.5761573344178369, 0.59...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
476,phase-encoding,angleradius,False,0.8,0.10,0.14290,"[-11.71491673358473, -8.11869271170008, 24.296...","[0.24875314351995803, 0.5761573344178369, 0.59...","[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
477,HSGS,angleradius,False,0.8,0.10,0.57140,"[-1.651246856480039, -0.0738426655821645, 6.99...","[0.24875314351995803, 0.5761573344178369, 0.59...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
478,phase-encoding,angleradius,True,0.8,0.10,0.42860,"[-24.542146055592518, -23.331908271927993, -41...","[0.24875314351995803, 0.5761573344178369, 0.59...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
experiment_data.to_csv('results/experiment_real_xor_v22.csv')